<a href="https://colab.research.google.com/github/rezamohammadloo/SignLanguageClassifier/blob/main/signLanguageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sign-language-mnist:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3258%2F5337%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240206%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240206T104333Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D86dcf7e3398773e8ac65dcb00e28a6bdb2fa50e3694f66bbe2c0628e490869be9335549d71c72fe957319b895d60fd5df22acda83e981a3e7e54b7b93892a5db006912f3bf13ff1f14c6cd0dc0766fe7411cb12e66aca01e9a3af61b36b8ccf857cc975a92c06193873359a4bf91f4b3461d503e7638fdcd9e5cea705702ab49635e05fdda7a6997587f0531cd1cc49b502b935b43b9f6fdc3377425bd8ef10dd2f63990529af5764e302ee687d61af2ae155f6cf97ca13478c5eee61a1242d951b53980ec7afd15a79c07b614b20f38a6908e36403d52c95bec70500bc8145e431ddc014662b6d73bb03319aab113b83ef45a2123cd685e6b77a39701007deb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


hello there, here we are going to creat a cnn to classify Sign Language.
![/kaggle/input/sign-language-mnist/amer_sign2.png](http://)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import matplotlib.pyplot as plt
from keras.optimizers import Adam

In [ ]:
train_data = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_data  = pd.read_csv('/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

In [ ]:
train_data.head()

In [ ]:
print("number of classes:", len(train_data['label'].unique()))

if we take a look at the data we will notice that the lables are saved in the first column, so we have to seperate them.

In [ ]:
X_train = train_data.copy()
X_test  = test_data.copy()
y_train = X_train.pop('label')
y_test  =  X_test.pop('label')

In [ ]:
print('X train shape :', X_train.shape)
print('train label shape:', y_train.shape)
print('X test shsape:', X_test.shape)
print('test label shape', y_test.shape)


let's see how many samples we have from each sign.

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=y_train.value_counts().index, y=y_train.value_counts().values)

it's fairly balanced data.

we should reshape our data to look like standard images
(28*28)

In [ ]:
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test  = X_test.values.reshape(-1, 28, 28, 1)

In [ ]:
X_train.shape

let's visualize some samples!

In [ ]:
#you can change image_number to see different images.(between 0 and 27454)
image_number = 27000
img = X_train[image_number, :, :]
img = img.reshape((28,28))
plt.imshow(img,cmap='gray')
plt.title( y_train[image_number])
plt.axis("off")
plt.show()

normalizing images and encoding labels.

In [ ]:
X_train = X_train / 255.0
X_test  = X_test / 255.0
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
from keras.utils import to_categorical # convert to one-hot-encoding
y_train = to_categorical(y_train, num_classes = 25)
y_test  = to_categorical(y_test,  num_classes = 25)

**defining the model.**

In [ ]:

from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense

model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation = 'relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2,2),

    Conv2D(32, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(25,  activation = 'softmax')])

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1)  # randomly shift images vertically 10%

datagen.fit(X_train)

In [ ]:
# Fit the model
epochs = 20
batch_size = 250
history = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test), steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
history.history.keys()

In [ ]:
plt.subplots(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.plot(history.history['loss'], color='r', label="train loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], color='b', label="accuracy")
plt.plot(history.history['val_accuracy'], color='r', label="val_accuracy")
plt.legend()

seems it's getting better and better and we don't have any obvious overfitting, so let's train another 10 epochs:

In [ ]:
history2 = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = 10, validation_data = (X_test,y_test), steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
plt.subplots(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history2.history['val_loss'], color='b', label="validation loss")
plt.plot(history2.history['loss'], color='r', label="train loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history2.history['accuracy'], color='b', label="accuracy")
plt.plot(history2.history['val_accuracy'], color='r', label="val_accuracy")
plt.legend()

still no problem, so let's give it another try!

In [ ]:
history3 = model.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = 10, validation_data = (X_test,y_test), steps_per_epoch=X_train.shape[0] // batch_size)

In [ ]:
plt.subplots(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history3.history['val_loss'], color='b', label="validation loss")
plt.plot(history3.history['loss'], color='r', label="train loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history3.history['accuracy'], color='b', label="accuracy")
plt.plot(history3.history['val_accuracy'], color='r', label="val_accuracy")
plt.legend()

strangely our net has always been working better on test than training(maybe it's because our test set is easier to classify, so it's better next time mix them and shuffle), but anyway we got a good result here and it could have performed even better  if we'd let it train longer. although we didn't use any regularization algorithm(like l2 or dropout) but our net didn't show any overfitting problem.


In [ ]:
overal_history = {}
for k in history.history.keys():
    overal_history[k] = history.history[k] + history2.history[k] + history3.history[k]

In [ ]:
plt.subplots(figsize=(20, 5))

plt.subplot(1, 2, 1)
plt.plot(overal_history['val_loss'], color='b', label="validation loss")
plt.plot(overal_history['loss'], color='r', label="train loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(overal_history['accuracy'], color='b', label="accuracy")
plt.plot(overal_history['val_accuracy'], color='r', label="val_accuracy")
plt.legend()